In [1]:
import numpy as np
import scipy as sci
import matplotlib.pyplot as plt
from jinja2 import Template
from pyne import data
from pyne.material import Material
from pyne import serpent
from pyne import nucname
import pdb

First we want the helium density in $10^24$ atoms per cc.

\begin{align}
PV&=nRT\\
\frac{n}{V}&=\frac{P}{RT}
\end{align}

In [2]:

R = 8.314 # j/k-mol
temp = [250 ,750] #celsius
pres = [6*10**6, 5.84*10**6] #pascals
hedensity = np.zeros(len(temp))


for i, t in enumerate(temp):
    hedensity[i] = ((pres[i])/((t+273.15)*R)) # mol/m^3 

hedensavg = np.sum(hedensity)/len(hedensity)*(6.022*10**-7)# 10^24/cc

print(hedensavg)

0.000622077124202638


### Isotopic composition of the fuel

The enrichment of the UCO kernels in the TRISO particles is known.  Additionally, we know the molar contentrations of the three compounds that make UCO: $UO_2$ , $UC$, and $UC_{1.86}$. The [PyNE](https://pyne.io/) python library is used to create the material cards for the fuel.
<br>
UCO is created from $UO_2$, $UC$, and $UC_{1.86}$ in molar concentrations of 0.714, 0.164, and 0.123, respectively.

In [3]:
leu = Material({'U235': 0.1975 , 'U238': 0.8025})
uo2 = Material()
uc = Material()
uc186 = Material()

uo2.from_atom_frac({'O16': 2.0 , leu :1.0})
uc.from_atom_frac({'C12':1, leu: 1.0})
uc186.from_atom_frac({'C12':1.86, leu : 1.0})

uco = Material()
uco.from_atom_frac({uo2: 0.714, uc: 0.164, uc186: 0.123})
print(uco.mcnp()) #the negative values for composition mean it is mass fraction, just like serpent


m?
     6012 -1.7770e-02
     8016 -8.6113e-02
     92235 -1.7698e-01
     92238 -7.1913e-01



/home/zoe/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  """Entry point for launching an IPython kernel.


In order to find the core dimensions, I took an image of the reactor core given by X-energy in a presentation to the NRC.  
<br>
Then, I opened it in an image viewer that gave the cursor's coordinates in pixels, measured the reactor core features in pixels.  Knowing the outer core diameter, developed a scale between pixels and meters, and used that to determine the core geometry.

\begin{align}
s (\frac{cm}{px}) = 100* \frac{RPV_D (m)}{RPV_D (px)}\\
x (cm) = x (px) * scale
\end{align}

In [4]:
core_dm = 4.88 # m
core_dpx = 69 #px
scale = core_dm/core_dpx *100 # cm/px

inner_corerad_px = 17.5
inner_corerad_cm = inner_corerad_px *scale
reflect_px = 13
reflect_cm = reflect_px *scale

print(2*(reflect_cm+inner_corerad_cm)) #can't be more than 488, which is the diameter including RPV
print()
print(inner_corerad_cm)
print(reflect_cm)
print(inner_corerad_cm+reflect_cm)

431.4202898550724

123.76811594202897
91.94202898550724
215.7101449275362


I found the midpoint of the cones on the top and bottom , and used the distance between these two midpoints as an adjusted height.  Then, I simplified by letting the reflector directly surround this region.

### Creating the input file

The first step to creating the input file is to create the pebble and particle location input cards.  This can be done using serpent's built in particle dispersal routine, which takes the dimensions of the particle, the volume they are dispersed in, and the particle number or packing fraction, the universe the particles are in, and creates a plain text file, where each line is:

\begin{align}
x-coordinate\ \ y-coordinate\ \  z-coordinate\ \  universe
\end{align}

In order to split the automatically generated distribution file into several sub-files for modeling different fuel compositions, the cell below splits a given text file into multiple.  It can also append a character to the universe name, if needed.

origin = open('pbbls-20.inp','r')

filenames = ['pbbls-20-0.inp','pbbls-20-1.inp','pbbls-20-2.inp','pbbls-20-3.inp','pbbls-20-4.inp','pbbls-20-5.inp','pbbls-20-6.inp']

lines = 0
for line in list(origin):
    lines += 1
origin.seek(0)
i = 0

file = ['f0','f1','f2','f3','f4','f5','f6']
for index, name in enumerate(filenames):
    file[index] = open(name,'a+')
subuniverse = ['a','b','c','d']    
    
while i < lines:
    for index, f in enumerate(file):
        #use lines below if you need to append a new character to universe name
        #newline = origin.readline()
        #f.write(newline[:-1] + subuniverse[index] + newline[-1:])
        f.write(origin.readline())
        i += 1
        if i == lines:
            break
for f in file:
    f.close()
origin.close()

After the above is used to create the input file, it is run in the terminal with Serpent 2.1.31

In [10]:
dep = serpent.parse_dep('../single-pebble/burn-20/htgr-mr-burn-20.inp_dep.m')

In [20]:
mats = dep['MAT_fuel_MATERIAL']
#mats[1].comp.keys
newdict = {nucname.serpent(k): v for k,v in mats[6].comp.items()}

for k,v in newdict.items():
    print('{}.08c'.format(k),"     " ,v)

H-1.08c       1.4039761314610683e-08
H-2.08c       8.575872266426048e-09
H-3.08c       1.7809450461743885e-07
He-3.08c       2.403414454964853e-09
He-4.08c       4.1936188654818196e-06
Li-6.08c       3.7024814426859424e-08
Li-7.08c       1.3328754919071045e-08
Be-9.08c       3.1081994308479828e-09
B-10.08c       2.3048449742374247e-09
B-11.08c       4.931884902411149e-10
C-nat.08c       0.017770521110676857
N-14.08c       2.1979529803730034e-12
N-15.08c       1.618496866254497e-09
O-16.08c       0.08611036021385192
O-17.08c       2.766667140376986e-08
Ca-46.08c       6.52385887434054e-26
Ca-48.08c       1.5542179587800361e-30
Ti-47.08c       1.0258065919017555e-23
Ti-48.08c       1.5385370706399525e-24
Ti-49.08c       1.8358918241640898e-17
Ti-50.08c       9.113288319649272e-15
Cr-52.08c       3.286683228976092e-14
Cr-53.08c       6.121212960079972e-14
Cr-54.08c       1.178486051483799e-13
Mn-55.08c       2.0837389940765826e-13
Fe-56.08c       4.405310855372948e-13
Fe-57.08c       1.19

In [7]:
serpent.parse_res('../full-core/htgr-20/htgr-20-full-core.inp_res.m')

{'np': <module 'numpy' from '/home/zoe/anaconda3/lib/python3.7/site-packages/numpy/__init__.py'>,
 'TOT_FLUX': array([[9.3077e+02, 3.5800e-03]]),
 'TOT_CUTRATE': array([[0., 0.]]),
 'PU239_CAPT': array([[0.0191427, 0.02283  , 0.0424034, 0.02234  ]]),
 'ANA_INV_SPD': array([[1.51219e-06, 2.35000e-03]]),
 'RES_MEMSIZE': array([30.69]),
 'URES_DILU_CUT': array([1.e-09]),
 'SRC_NORM_MODE': array([2]),
 'TOT_RR': array([[3.1922e+02, 4.5900e-03]]),
 'ANA_EALF': array([[1.78105e-07, 1.31000e-02]]),
 'TOT_NUCLIDES': array([316]),
 'PU240_FISS': array([[1.91763e-05, 7.03540e-01, 4.48023e-05, 7.03600e-01]]),
 'ADJ_NAUCHI_LIFETIME': array([[0.00100877, 0.00856   , 0.00100924, 0.00871   , 0.00082817,
         0.12952   ]]),
 'TOT_SF_RATE': array([0.]),
 'ANA_THERM_TIME': array([[0.00182407, 0.00564   , 0.0018247 , 0.00566   , 0.00169437,
         0.08766   ]]),
 'TOT_CELLS': array([4]),
 'ELECTRON_DECAY_SOURCE': array([0.]),
 'VERSION': array([b'Serpent 2.1.31'], dtype='|S14'),
 'REA_SAMPLING_EFF'

In [22]:
fuel = (4/3)*np.pi*(0.02125**3)*(18000)
pyroc = 18000*( ((4/3)*np.pi*(0.03475**3) - 
                 (4/3)*np.pi*(0.03075**3))
         + ( (4/3)*np.pi*(0.04225**3) - 
            (4/3)*np.pi*(0.03828**3) ))
sic = ((4/3)*np.pi*(0.03825**3) - (4/3)*np.pi*(0.03475**3))*18000

graph = ((4/3)*np.pi*(3.0**3)) - fuel - pyroc - sic

tot = (4/3)*np.pi*(3.0**3)

In [23]:
print(fuel/tot)
print(pyroc/tot)
print(sic/tot)
print(graph/tot)

0.006397135416666668
0.021474480715333345
0.009332895833333325
0.9627954880346667


In [24]:
(fuel+pyroc+sic+graph)/tot

1.0

$$Bg^2 = \frac{( \upsilon\Sigma_f -\Sigma_a )}{D}$$
$$Power Density = \frac{kW}{g_{fuel}} $$
$$ 0.11 {kW/g} = \frac{20*1000}{m_{fuel}}$$
$$m_{fuel} = 181818.18 g$$
$$m_{fuel}/\rho = 181818.18/11.0 = 16528.92 cc$$
$$ V_{p} = 18,000 * 4/3 * \pi * 0.02125^{3} = 0.7235$$
$$ 16528.92 cc / 0.7235 = # pebb = 22846 $$
$$ \theta = \frac{tot pebb vol}{inner core vol} = 0.56 (natural pacfrac)$$
$$ inner core vol = 4613967.37 = v = H * \pi * R^{2} $$
$$ H = \frac{V}{\pi*R^{2}} $$
$$ Bg^{2} = (\frac{\pi}{H})^{2} + (\frac{2.405}{R})^{2} = \frac{( \nu\Sigma_f -\Sigma_a )}{D} $$
$$(\frac{\pi^{2}R^{2}}{v})^{2} + (\frac{2.405}{R})^{2} = \frac{( \nu\Sigma_f -\Sigma_a )}{D} = 9.98^{-5}$$
$$(\frac{\pi^{4}}{v^{2}})* R^{4} + (\frac{2.405}{R})^{2} = \frac{( \nu\Sigma_f -\Sigma_a )}{D} = 9.98^{-5}$$

((clean up variable names))

In [2]:
4**2

21288694891424.72

$$ Bg^{2} = (\frac{\pi}{H})^{2} + (\frac{2.405}{R})^{2} = \frac{( \nu\Sigma_f -\Sigma_a )}{D} $$
$$(\frac{\pi}{H})^{2} + (\frac{2.405}{R})^{2} = \frac{( \nu\Sigma_f -\Sigma_a )}{D} = 9.98^{-5}$$
$$(\frac{\pi}{H})^{2} + (\frac{2.405}{R})^{2} = 9.98^{-5}$$